In [5]:
import requests as re
from bs4 import BeautifulSoup
from os import path
import numpy as np
import pandas as pd

import xml.etree.ElementTree as ET
import datetime
from datetime import datetime as extra_datetime
import time

# save path for different users
PATH_DIR = r'C:\Users\Никита\Андан\Project\data\macro'
parent_dir = r"/Users/andrewnizov/Desktop/АнДан/Проект"  

There is a lot of hardcoding for parsing files with macrodata. Be cautious about changing anything

## Precious metals prices parser
CBR XML

* 1 - gold
* 2 - silver
* 3 - platinum
* 4 - palladium

In [ ]:
def parse_precious_metals_rates():
    '''
    This function is needed for parsing prices of 4 precious metals for the studied period of time.
    '''
    # Dates in DD/MM/YYYY format
    start_date = datetime.date(2010, 1, 1).strftime('%d/%m/%Y')
    end_date = datetime.date.today().strftime('%d/%m/%Y')
    
    # URL creating for the further request
    url_1 = f'https://www.cbr.ru/scripts/xml_metall.asp?date_req1={start_date}&date_req2={end_date}'
    response_precious = requests.get(url_1)
    
    tree_precious_rate = BeautifulSoup(response_precious.content, 'html.parser')
    
    dates = []
    gold_rates = []
    silver_rates = []
    platinum_rates = []
    palladium_rates = []
    
    # Forms five sets with dates and prices of 4 precious metals, that will be used for final dataframe
    helpa = 1
    for stat in tree_precious_rate.find_all('record'):
        if helpa == 1:
            gold_rates.append(float(stat.buy.text.replace(',', '.')))
            dates.append(extra_datetime.strptime(stat.get('date'), '%d.%m.%Y').date().strftime('%d.%m.%Y'))
        elif helpa == 2:
            silver_rates.append(float(stat.buy.text.replace(',', '.')))
        elif helpa == 3:
            platinum_rates.append(float(stat.buy.text.replace(',', '.')))
        else:
            palladium_rates.append(float(stat.buy.text.replace(',', '.')))

        if helpa == 4:
            helpa = 0

        helpa += 1

    precious_metals = pd.DataFrame({'gold_rates': gold_rates, 'silver_rates': silver_rates, 'platinum_rates': platinum_rates, 'palladium_rates': palladium_rates}, index=pd.to_datetime(dates))
    final_precious_metals = precious_metals.sort_index()
    
    return final_precious_metals
    
    # Code saves the result to CSV file
    '''final_precious_metals.to_csv(path.join(parent_dir, 'precious_metals', 'data_precious_metals.csv'))'''

In [ ]:
parse_precious_metals_rates()

In [ ]:
gold_price = parse_precious_metals_rates()['gold_rates']
silver_price = parse_precious_metals_rates()['silver_rates']
platinum_price = parse_precious_metals_rates()['platinum_rates']
palladium_price = parse_precious_metals_rates()['palladium_rates']
x = gold_price.index

In [ ]:
fig, ax = plt.subplots(figsize = (12, 6))
plt.plot(x, palladium_price)
plt.show()

## Interbank market rates parser
CBR XML

**Code = 3: Weighted average actual rates on ruble loans provided by Moscow banks.**

In [ ]:
def parse_interbank_market_rates():
    '''
    This function is needed for parsing dynamics of interbank credit market rates for the studied period of time.
    '''
    # Dates in DD/MM/YYYY format
    start_date = datetime.date(2010, 1, 1).strftime('%d/%m/%Y')
    end_date = datetime.date.today().strftime('%d/%m/%Y')
    
    # URL creating for the further request
    url_2 = f'https://www.cbr.ru/scripts/xml_mkr.asp?date_req1={start_date}&date_req2={end_date}' 
    response_credit_rate = requests.get(url_2)
    
    tree_credit_rate = BeautifulSoup(response_credit_rate.content, 'html.parser')
    
    dates = []
    interbank_credit_market_rates = []
    
    # Forms two sets with dates and exchange rates, that will be used for final dataframe
    # Code appeals to elusive html element, using xml.etree.ElementTree
    for line in tree_credit_rate.find_all('record'):
        root = ET.fromstring(str(line))
        if root.attrib.get('code') == '3':
            dates.append(extra_datetime.strptime(line.get('date'), '%d/%m/%Y').date().strftime('%d/%m/%Y'))
            if line.c1.text != '-':
                interbank_credit_market_rates.append(float(line.c1.text))
            else:
                interbank_credit_market_rates.append('-')
    
    interbank_rates = pd.DataFrame(data=interbank_credit_market_rates, index=pd.to_datetime(dates), columns=['Interbank credit market rates'])
    interbank_rates_final = interbank_rates.sort_index()
    
    # Code saves the result to CSV file
    interbank_rates_final.to_csv(path.join(parent_dir, 'interbank_credit_market_rates', 'data_interbank_rates.csv'))

In [ ]:
parse_interbank_market_rates()

## Russian statisical service parsing

In [27]:
def get_GDP_excel(save_path):
    '''
    save_path: str or path-object - path to save xlsx GDP

    returns
    saves GDP (quartely data) as xlsx file
    '''

    # getting link from rosstat website
    responce = BeautifulSoup(re.get('https://rosstat.gov.ru/statistics/accounts').content, 'html.parser')
    link = responce.find_all('a', {'class': 'btn btn-icon btn-white btn-br btn-sm'})[1].get('href')

    responce = re.get(r'https://rosstat.gov.ru' + link)
    with open(path.join(save_path, 'GDP.xlsx'), 'wb') as file:
        file.write(responce.content)
    

In [28]:
get_GDP_excel(PATH_DIR)

In [6]:
# looking at quartely data
df = pd.read_excel(path.join(PATH_DIR, 'GDP.xlsx'), 9)
df

,К содержанию,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51
0,"Валовой внутрений продукт 1)\n(в ценах 2021г.,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011,NaN,NaN,NaN,2012,NaN,NaN,NaN,2013,NaN,...,NaN,NaN,20222),NaN,NaN,NaN,20232),NaN,NaN,NaN
2,I квартал,II квартал,III квартал,IV квартал,I квартал,II квартал,III квартал,IV квартал,I квартал,II квартал,...,III квартал,IV квартал,I квартал,II квартал,III квартал,IV квартал,I квартал,II квартал,III квартал,IV квартал
3,26470.295854,28624.554577,30177.235769,32311.819119,27979.628161,30013.804133,31230.625524,33091.525158,28296.087789,30530.548807,...,34818.893245,38142.716608,31103.95782,31679.223218,33857.522587,37439.930767,30615.796202,33291.76746,35797.406698,39268.666618
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"1) Данные содержат изменения, связанные с внед...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2) Без учета статистической информации по Доне...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
def process_gdp(path):
    '''
    path: str or path-object - path to find .xlsx file with GDP

    returns:
    pd.DataFrame of real GDP
    '''
    # Real GDP in 2021 prices (quarterly)
    gdp_raw = pd.read_excel(path, 9)
    
    df = pd.DataFrame({})
    df['gdp in 2021 prices'] = gdp_raw.iloc[3,:].astype('float32').to_numpy()

    # Real GDP in 2021 prices without seasonality (quarterly)
    gdp_raw = pd.read_excel(path, 10)
    df['gdp without seasonality in 2021 prices'] = gdp_raw.iloc[3,:].astype('float32').to_numpy()
    
    
    df['year'] = 2011 + np.arange(df.shape[0]) // 4
    df['quater'] = 1+np.arange(df.shape[0]) % 4

    gdp_raw = pd.read_excel(path, 10)
    df['gdp without seasonality in 2021 prices'] = gdp_raw.iloc[3,:].astype('float32').to_numpy()

    return df

In [24]:
# saving as .csv file in PATH_DIR
df = process_gdp(path.join(PATH_DIR, 'GDP.xlsx'))
df.to_csv(path.join(PATH_DIR, 'GDP_processed.csv'), \
          index = False, encoding = 'utf-8')
df.isna().sum()

gdp in 2021 prices                        0
gdp without seasonality in 2021 prices    0
year                                      0
quater                                    0
dtype: int64

## Central bank of Russia parsing macro data

In [9]:
# links to get .xlsx files
LINKS = ['https://www.cbr.ru/vfs/statistics/households/households_bm.xlsx', \
         'https://www.cbr.ru/vfs/statistics/nfs_hd/nonfinancial_sector_households_debt_extended.xlsx',\
        'https://www.cbr.ru/Content/Document/File/128613/monetary_agg.xlsx']

for link in LINKS:
    res = re.get(link)
    with open(path.join(PATH_DIR, link.split('/')[-1]), 'wb') as file:
        if res.status_code == 200:
            print('OK')
            file.write(res.content)       

OK
OK
OK


In [10]:
def preprocess_balance(path):
    '''
    path: str or path-object - path to find .xlsx file with balance sheet

    returns:
    pd.DataFrame balance sheet of households (monthly data)
    '''

    # Reading .xlsx file and processing it (hardcode)
    raw_df = pd.read_excel(path, 'Балансы')
    df = pd.DataFrame({})

    columns = ['Наличная национальная валюта', 'Наличная иностранная валюта', 'Депозиты', \
               'Денежные средства на брокерских счетах -всего(5)', 'Долговые ценные бумаги', \
               'Котируемые акции и паи и акции инвестиционных фондов*', 
              'Кредиты кредитных организаций - всего']
    dates = pd.to_datetime(raw_df.loc[2,:].drop('Unnamed: 0'))
    df['date'] = dates
    
    for column in columns:
        df[column] = raw_df[raw_df['Unnamed: 0'] == column].drop('Unnamed: 0', axis = 1).astype('float32').transpose()
    
    df = df.reset_index().drop('index', axis = 1)

    df.rename({'Денежные средства на брокерских счетах -всего(5)': 'Денежные средства на брокерских счетах', 
              'Котируемые акции и паи и акции инвестиционных фондов*': 'Котируемые акции и паи и акции инвестиционных фондов', 
              'Кредиты кредитных организаций - всего': 'Кредиты кредитных организаций'}, axis = 1, inplace = True)
    
    return df

In [25]:
df = preprocess_balance(path.join(PATH_DIR, 'households_bm.xlsx'))
df.to_csv(path.join(PATH_DIR, 'households_bm_processed.csv'), \
         index = False, encoding = 'utf-8')

df.isna().sum()

date                                                    0
Наличная национальная валюта                            0
Наличная иностранная валюта                             0
Депозиты                                                0
Денежные средства на брокерских счетах                  0
Долговые ценные бумаги                                  0
Котируемые акции и паи и акции инвестиционных фондов    0
Кредиты кредитных организаций                           0
dtype: int64

In [12]:
def process_moneraty_mass(path):
    '''
    path: str or path-object - path to find .xlsx file with monetary mass

    returns:
    pd.DataFrame balance of monetary supply (monthly data)
    '''
    raw_df = pd.read_excel(path, 'Денежные агрегаты')
    df = pd.DataFrame({})
    
    columns = ['Денежный агрегат М0', 'Денежный агрегат М1', 'Денежный агрегат М2', 'Широкая денежная масса']
    for column in columns:
        df[column] = raw_df[raw_df['Денежные агрегаты*, млрд руб.'] == column].iloc[:,97:].transpose().astype('float32')

    df = df.reset_index().rename({'index':'date'}, axis = 1)
    df['date'] = pd.to_datetime(df['date'])

    return df

In [26]:
df = process_moneraty_mass(path.join(PATH_DIR, 'monetary_agg.xlsx'))
df.to_csv(path.join(PATH_DIR, 'monetary_agg_processed.csv'),\
         index = False, encoding = 'utf-8')

df.isna().sum()

date                      0
Денежный агрегат М0       0
Денежный агрегат М1       0
Денежный агрегат М2       0
Широкая денежная масса    0
dtype: int64

In [15]:
def process_debt(path):
    '''
    path: str or path-object - path to find .xlsx file debt indicators

    returns:
    pd.DataFrame balance of debt indicators (quarterly data)
    '''

    # Reading .xlsx indicators of household and non-financial sector creditworthiness
    raw_df = pd.read_excel(path)
    column_names = raw_df.iloc[4:15, 1:].transpose().iloc[0,:]
    
    df = raw_df.iloc[4:15, 1:].transpose()\
        .set_axis(column_names, axis = 1) \
        .drop('Unnamed: 1').astype('int32')
    df['date'] = pd.to_datetime(raw_df.iloc[3,2:].transpose())
    df = df.reset_index().drop('index', axis = 1)

    return df


In [19]:
df = process_debt(path.join(PATH_DIR, 'nonfinancial_sector_households_debt_extended.xlsx'))
df.to_csv(path.join(PATH_DIR, 'nonfinancial_sector_households_debt_extended_processed.csv'), \
          index = False, encoding = 'utf-8')
df.isna().sum()

Unnamed: 1
 Долг нефинансового сектора и домашних хозяйств, итого                           0
    Нефинансовый сектор                                                          0
          Внутренние заимствования                                               0
              Кредиты                                                            0
              Долговые ценные бумаги в портфеле резидентов                       0
          Внешние заимствования                                                  0
              Кредиты и займы, полученные от нерезидентов                        0
              Долговые ценные бумаги в портфеле нерезидентов, включая векселя    0
    Домашние хозяйства и НКООДХ                                                  0
          Внутренние заимствования                                               0
          Внешние заимствования                                                  0
date                                                                        